In [14]:
'''

2019. 11. 28.

by Yumi Kim CBBL at GNU
Screening file and Suggest Candidates which has good activity values

'''

import pandas as pd
import numpy as np

def remove_specific_char(origine_srt, specific_char):
    return ''.join( c for c in origine_srt if  c not in specific_char )


def get_weight_list(weights):
    weights = remove_specific_char(weights, '[]')
    weights = weights.split(" ")

    weight_list = []
    
    for i in range(len(weights)):
        if(len(weights[i]) != 0):
            weight_list.append(float(weights[i]))
    
    return weight_list

KCB_data_path = "kcb_data/"
KCB_data_list = ['KCB_1_8000.csv','KCB_4_32000.csv','KCB_5_40000.csv','KCB_6_48000.csv',
                 'KCB_7_56000.csv','KCB_8_64000.csv','KCB_9_72000.csv','KCB_10_80000.csv',
                 'KCB_11_88000.csv','KCB_12_96000.csv','KCB_14_112000.csv','KCB_15_120000.csv',
                 'KCB_17_136000.csv','KCB_20_157694.csv']
KCB_data_group_num=14

file_name_descriptor_list = 'qsar_descriptor_list.csv'
descriptor_list_df = pd.read_csv(file_name_descriptor_list)

file_name_all_qsar_df = 'qsar_model_list.csv'
all_qsar_df = pd.read_csv(file_name_all_qsar_df)

## Set activity value
if(input_IC50_value > 10):
    input_IC50_value = 10
    
# Input parameter
input_IC50_value = 0.3
screening_output_file_name = 'screening_result.csv'

# Get QSAR Weights and bias
bias = all_qsar_df['bias_vals'].values[0]
bias = remove_specific_char(bias, '[]')
bias = float(bias)
weights = all_qsar_df['weights_vals'].values[0]

weight_list = get_weight_list(weights)

des_list = ['smile','corp_id']
des_list_except_smile = []
for i in range(descriptor_list_df.shape[0]):
    des_list.append(descriptor_list_df['0'][i])
    des_list_except_smile.append(descriptor_list_df['0'][i])

result_df = pd.DataFrame()

for i in range(KCB_data_group_num):
    file_name_KCB_data = KCB_data_list[i]
    data_df = pd.read_csv(KCB_data_path+file_name_KCB_data)

    optimized_df = data_df[des_list]

    # Calculate IC50 value with QSAR model
    optimized_df["Pred_IC50"] = 0
    for i in range(len(weight_list)):
        print(optimized_df[des_list_except_smile[i]]*weight_list[i])
        optimized_df["Pred_IC50"] += optimized_df[des_list_except_smile[i]]*weight_list[i]
    optimized_df["Pred_IC50"]+= bias

    optimized_df = optimized_df.sort_values(["Pred_IC50"], ascending=[True])

    # Selection with Condition
    low_value = optimized_df['Pred_IC50'] > 0
    high_value = optimized_df['Pred_IC50'] < input_IC50_value
    same_bias = optimized_df['Pred_IC50'] != bias
    suggest_compounds_df = optimized_df[low_value & high_value & same_bias]
    if(i == 0):
        result_df=suggest_compounds_df
    else:
        result_df=result_df.append(suggest_compounds_df)
        
result_df.to_csv(screening_output_file_name, index=False)



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1144,1145,1223,1224,1225,1226,1227,1228,1445) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0      -97.960291
1      -66.981283
2       -9.160850
3      -26.583651
4      -64.621670
5      -18.579474
6      -23.397843
7       43.557002
8      -55.897050
9      -72.447389
10      54.317366
11      -4.930732
12      17.436020
13      47.555785
14     -36.233609
15      -9.630129
16      -0.727052
17      42.314404
18      38.864214
19       6.926819
20     -49.492387
21     -43.206695
22     -14.296479
23     -99.539976
24     -47.806949
25     -11.586559
26     -36.524430
27     -81.409953
28      19.478374
29     -10.707487
          ...    
7748    36.359191
7749    37.066413
7750    70.087776
7751    37.066413
7752    48.335714
7753   -48.831431
7754    54.026545
7755    67.206008
7756    23.272261
7757    61.482129
7758    29.009360
7759    28.381451
7760    22.783154
7761    61.482129
7762    54.667672
7763    54.575138
7764    16.775064
7765    49.247833
7766    49.247833
7767   -16.404928
7768    16.067841
7769     9.841635
7770    34.911697
7771    41.726153
7772    25

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

0       -17.138590
1        12.452411
2        97.226630
3      -129.501113
4         2.921426
5       -87.127222
6       -22.736887
7        75.824874
8       -84.794047
9       -40.179516
10       76.862575
11      -56.505130
12       27.363579
13       46.663495
14       -4.441624
15       13.219120
16      -12.888642
17      -49.922008
18       63.557531
19      -13.602475
20       71.720337
21        8.096711
22      183.573924
23       20.615218
24      164.538391
25      139.481548
26      158.517082
27       39.802771
28       62.506611
29       64.535746
           ...    
7192   -136.804677
7193   -130.043096
7194     37.449768
7195     -8.407361
7196   -185.999633
7197   -185.999633
7198    -87.609720
7199    -73.643719
7200   -185.999633
7201     -7.898424
7202    -29.353057
7203    -48.758725
7204    -50.900223
7205    -26.933958
7206    -79.360989
7207    108.634731
7208     54.304146
7209    -65.077729
7210     18.718274
7211    -55.288971
7212   -103.545370
7213     70.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

...

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (67,121,185,186,187,188,189,190,191,192,248,249,250,251,252,253,254,255,785,786,822,823,824,826,828,831,843,845,850,854,1023,1024,1060,1061,1062,1064,1066,1069,1081,1083,1088,1092,1140,1141,1143,1145,1148,1151,1182,1183,1184) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

0       164.194694
1        20.668095
2       -11.897208
3       -48.963622
4       243.641607
5        19.075191
6        16.206642
7         7.528289
8       267.640920
9       104.206326
10        3.595601
11     -208.967855
12       26.828205
13       31.620136
14      207.983030
15      219.060653
16      -10.813240
17      221.274856
18      -17.574821
19      163.546957
20      -74.238580
21        2.921426
22       70.563664
23      -34.554781
24      -22.736887
25      147.512164
26      -40.648795
27      -44.072547
28      -29.921479
29        1.784581
           ...    
7970    -10.793412
7971     32.069586
7972     12.644089
7973     65.547008
7974     78.382774
7975     26.087934
7976     -0.409793
7977    -43.081113
7978   -144.095021
7979   -127.696703
7980    -86.981812
7981    146.243128
7982    -47.469861
7983      5.122409
7984    141.001747
7985     17.984613
7986    -60.431209
7987     29.227475
7988     52.664975
7989     26.834814
7990     82.877275
7991     47.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (67,121,185,186,187,188,189,190,191,192,248,249,250,251,252,253,254,255,785,786,789,790,791,792,793,795,822,823,824,826,828,831,834,842,843,845,848,849,850,851,908,909,910,911,912,914,1023,1024,1060,1061,1062,1064,1066,1069,1072,1080,1081,1083,1086,1087,1088,1089,1140,1141,1143,1145,1147,1148,1149,1150,1151,1152,1182,1183,1184) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: SettingWithCopyWar

0       100.062132
1        21.547166
2       -13.582646
3        64.059857
4        99.222717
5       133.592430
6       211.175448
7       104.133621
8       159.865432
9       145.800288
10      118.899378
11        6.331959
12       41.461771
13       81.257933
14      127.610778
15      185.603059
16      111.919682
17       98.072654
18       50.926661
19      120.214680
20      124.405142
21      174.888962
22      214.334817
23      194.129392
24      114.008303
25       76.822918
26      156.342536
27      130.571861
28      128.998786
29       98.323817
           ...    
5664     23.391233
5665     30.166033
5666     30.503120
5667     18.341529
5668      1.963039
5669     40.972664
5670    -34.396151
5671     11.857551
5672     21.157202
5673     29.405933
5674    -32.373626
5675    -38.758461
5676     21.342270
5677     85.831748
5678    -28.123679
5679     42.254918
5680    -17.422801
5681     -0.832805
5682     23.880341
5683     40.470337
5684    -42.360671
5685    -25.

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
suggest_compounds_df

,smile,corp_id,ALogP,AATS4m,AATS4v,C1SP3,LipoaffinityIndex,Pred_IC50
2597,O=[S](=O)(C=3C(OC([H])([H])C([H])([H])N1/N=C(/...,"307,061",-0.4294,97.546261,195.404830,3,3.248548,0.046905
2535,O=[S](=O)(C=2C(OC([H])([H])C(=O)N([H])C=1/N=C(...,"306,740",-0.5581,121.904850,221.304239,1,4.076750,0.118707
5519,O=C1N4/C(=C(/[H])C([H])=C1[H])[C@]5([H])C([H])...,"417,625",-1.9555,51.709633,157.658947,3,5.324443,0.185621


In [16]:
result_df

,smile,corp_id,ALogP,AATS4m,AATS4v,C1SP3,LipoaffinityIndex,Pred_IC50
3909,O=C2N(C=1N=C(N(C=1C(=O)N2C([H])([H])[H])C([H])...,"41,290",-0.2068,93.054804,192.368438,2,5.219208,0.162045
6881,O=C2/N=C(/N([H])C(=N/C1=C(\[H])C(=C([H])C([H])...,"44,625",-0.6813,60.178743,157.598796,2,4.578300,0.216549
207,O=C1ON=C(N1C([H])([H])C(=O)N([H])C2=C([H])C(=C...,"33,291",-0.7031,51.599535,161.999285,5,8.715344,0.267704
7835,[S]=C(N([H])C1=C([H])C(=C([H])C([H])=C1[H])C([...,"52,826",1.1761,82.184119,174.935483,2,5.655628,0.123604
5649,O=C3O[C@@](C=1N=C([S]C=1[H])N([H])C=2C([H])=C(...,"50,640",0.3804,64.781772,164.655661,5,5.816672,0.230353
3921,[H]/C(=N\C1=N\C2=C(N1C([H])([H])C([H])([H])C([...,"48,912",-1.2826,49.160855,166.050047,2,12.465608,0.263383
623,O=[S](=O)(C=2C=1C([Cl])=C([H])C([H])=C([H])C=1...,"53,614",0.4100,125.932201,226.015902,1,6.323917,0.123999
1387,[H][C@@]2(OC([H])([H])C([H])([H])C1=C([H])C([H...,"54,579",0.5567,49.298224,164.421618,5,13.645005,0.135701
4159,[F]C([F])([F])C1=C([H])C([H])=C(C([H])=C1[H])C...,"58,495",1.4257,57.567584,168.904743,8,11.520826,0.212805
2803,O=C2/N=C(/N([H])C(=N/C1=C(\[H])C(=C([H])C([H])...,"55,995",-0.6813,60.178743,157.598796,2,4.578300,0.216549
